In [ ]:
import keras
import pandas as pd
from glob import glob                                # Parcourir dossier de dataset
import matplotlib.pyplot as plt
import cv2                                           # traitement des images du dataset
from keras.datasets import mnist                     # Importe le dataset de mnist
from keras.utils import to_categorical               # pour modifier les labels
from keras.models import Sequential, load_model      # type de modele
from keras.layers import Dense, Flatten              # Fully Connected Layers
from keras.layers import Conv2D, MaxPooling2D        # Convolutional layers + Maxpooling layers
from keras.optimizers import Adam
from keras.losses import categorical_crossentropy    # Loss function

# from keras.callbacks import TensorBoard

import h5py                                          # Pour pouvoir enregistrer son modele en .h5

# Detection of handwritten digits with MNIST

MNIST est un exemple très souvent utilisé pour commencer avec les CNN, son dataset existe sur keras et peut être directement importé avec la fonction **load_data( )**

Note :
- Le dataset de MNIST a 60 000 exemples d'entrainement et 10 000 exemples de training
- Le format de chaque image est de 28x28 pixels

## STEP 1 : Load data

In [ ]:
# Les données sont déjà séparées en training set et testing set sur keras

(x_train, y_train), (x_test, y_test) = mnist.load_data()

Vous pouvez voir à quoi ressemble votre dataset avec un print. A noter que votre x_train est une liste [ ] de 60 000 exemples et que votre x_test est une liste de 10 000 exemples

- Chaque exemple se compose de 28 listes de 28 valeurs (car 28x28 pixels), il y a donc 28x28 = 784 valeurs, une par pixel
- Chaque pixel a une valeur comprise entre 0 et 255, car un int = 8 octets donc 2^8 = 256 valeurs possibles, mais on commence à zéro donc 0-255
- Les labels sont une valeur unique comprise entre 0 et 9 et correspondent au digit de l'image en question  
  
Vous pouvez verifier la shape de votre dataset (x_train, x_test, y_train et y_test)

## STEP 2 : Prepare Data


Ensuite, une fois les données importées, nous devons les préparer afin qu'elles puissent être données à notre modèle  
Dans un premier temps, nous allons ajouter une dimension à la fin de nos x_train et x_test pour préciser le nombre de canaux que nous utiliserons grâce à la fonction **.reshape( )**

- 1 pour Noir et Blanc
- 3 pour RGB
- L pour nuances de gris

Pour cet exemple, nous nous contenterons d'une image en N&B, donc la valeur de notre dimension supplémentaire sera égale à 1

In [ ]:
x_train = 
x_test = 

Nous allons ensuite normaliser notre dataset afin que les valeurs de nos pixels soient comprises entre 0 et 1 plutôt qu'entre 0 et 255. Cela facilitera les calculs pour la machine.

Pour ce faire, il faudra d'abord "recaster" nos variables *x_train* et *x_test* en tant que float.

In [ ]:
# Recast

x_train = 
x_test = 

# Normalize

x_train /= 
x_test /= 

Maintenant qu'on a bien modifié nos training et testing sets, il nous reste à modifier nos labels de façon à ce que la machine puisse mieux les comprendre.

Pour l'instant, notre y_train est une liste de 60 000 valeurs entre 0 et 9, qui sont les labels des images correspondantes. 

*y_train[8] = label de la 9ème image de notre dataset, x_train[8]*

Plutôt que d'avoir un label qui soit une valeur comprise entre 0 et 9, on va préférer avoir un label qui a la forme d'un one-hot vecteur. C'est à dire un vecteur composé de 10 valeurs (0 ou 1), avec un 1 a la position correspondante au label, et des 0 partout ailleurs.

*Exemple : on veut transformer un* **y_train[8] = 1** en **y_train[8] = [0, 1, 0, 0, 0, 0, 0, 0, 0, 0]**

On va utiliser une fonction de keras appelée **to_categorical(a, b)** qui prend en paramètre :
- a = la liste qu'on cherche à changer
- b = le nombre de sorties que nous avons

Note : Cette fonction saura automatiquement où placer le 1 dans votre vecteur. Vous devez donc juste lui donner le nombre de catégories possibles que notre modèle pourra avoir.

In [ ]:
y_train = 
y_test = 

## STEP 3 : Create Model

Maintenant que nos données sont prêtes, nous allons pouvoir créer notre modèle. Keras a énormément de fonctions qui nous permettent d'éviter d'avoir à recoder nos fonctions de loss ou de backpropagation, pour l'objectif de cet exercice, nous allons donc les utiliser :)

Nous allons commencer par définir une variable en tant que modèle en utilisant la méthode Sequential de keras. Utiliser cette méthode nous facilitera le travail pour ajouter nos couches et train notre modèle. Pour plus d'infos vous pouvez checker la doc

https://keras.io/models/sequential/

Nous allons ensuite pouvoir y ajouter les couches dont nous avons besoin. Rappelez vous l'ordre des couches (D'abord les couches de conv, puis les couches de Maxpool, puis les Fully Connected)

Pour ajouter une couche, vous pouvez utiliser la fonction **.add( )**


Pour chaque couche de convolution, vous devrez préciser :  
*Attention ! Pour la première couche, n'oubliez pas qu'il faut préciser l'* **input_shape**
- Le nombre de neurones
- La taille de kernel si couche conv
- La taille de Pool si couche MaxPool
- La fonction d'activation (nous utiliserons **'relu'**)

Pour chaque couche de MaxPooling, vous devrez préciser :
- La taille de Pool si couche MaxPool

Pour chaque couche de Fully Connected, vous devrez préciser :
- Le nombre de neurones
- La fonction d'activation (nous utiliserons **'relu'**)
*Attention ! Pour la dernière couche de FC, n'oubliez pas qu'il faut préciser le* **nombre de sorties et non le nombre de neurones** *et que votre fonction d'activation est* **'softmax'** *et non* **'relu'**

Voici la doc qui vous aidera à mieux comprendre Sequential https://keras.io/getting-started/sequential-model-guide/

Vous pouvez utiliser la fonction **.summary( )** pour voir les détails de vos couches

In [ ]:
# Define model
model = Sequential()

# Ajouter les couches de convolution



# Ajouter les couches de MaxPooling



# Alterner entre couche de convolution et maxpooling



# Applatir le dataset en un seul array
model.add(Flatten())



# Ajouter les couches Fully Connected



# Ajouter la dernière couche FC, mais avec cette fois le nombre de sorties au lieu du nombre de neurones et 'softmax' comme activation


# Compilation du modèle avant l'entrainement
model.compile(loss='categorical_crossentropy', 
              optimizer=Adam(), 
              metrics=['accuracy'])

# Recap de votre modèle, avec l'output shape à chaque couche
model.summary()

## STEP 4 : Train Model and save it

Nous devons maintenant entrainer notre modèle afin de voir ses performances. Pour cela, vous pouvez utiliser la fonction **.fit( )** qui prend en paramètres :
- x_train
- y_train,
- votre taille de batch (favorisez les puissances de 2)
- votre nombre d'epochs
- votre proportion de validation (par ex 0.2 pour 20%)

In [ ]:
training = model.fit(...)

# On évalue ensuite notre modèle sur notre testing set
score = model.evaluate(x_test, y_test, batch_size=128)
print('score = ', score)

# Save le modèle
model.save("votre_model.h5")

Vous pouvez voir l'évolution de vos loss et accuracy grâce à cette fonction (Kdo)

In [ ]:
historydf = pd.DataFrame(training.history, index=training.epoch)
historydf.plot(ylim=(0,1))

## STEP 5 : Testing our model

Maintenant que nous avons un modèle, nous allons pouvoir le tester. Pour cela, nous allons créer nos propres images de digits en blanc sur noir en allant sur ce site : https://www.piskelapp.com/
- Cliquer sur *Create Sprite* en haut à droite
- Dans les paramètres, resize l'image à 28x28
- N'oubliez pas de faire un **fond noir**
- Dessinez le digit que vous voulez tester **en blanc** (n'hésitez pas à en faire plusieurs)
- Exportez votre image (icone de la montagne)
- Vérifiez que vous êtes bien sous l'onglet PNG, et cliquez sur *Download Selected frame export*

Ensuite, une fois toutes vos images téléchargées, renommez les avec leur label correspondant (ce sera plus simple de vérifier le résultat), ex : image de 0 sera nommée 0.png
  
Mettez toutes vos images dans le même dossier que votre jupyter notebook
  
Pour aller chercher les images à tester, nous allons utiliser un module appelé **glob** qui se situe déjà dans vos imports.
  
Il faudra ensuite faire les mêmes modifications que nous avons faites à notre dataset dans le *STEP 2 : Prepare dataset* (reshape, normalize)

In [ ]:
# Recupérer toutes nos images
all_images = glob('*.png')

model = load_model('votre_model.h5')

# Loop in all_images to process each image
for image in all_images :
    # Open the image using cv2
    img = cv2.imread(image, 0)
    
    # Print the image to see (will only print the last one of the loop)
    #plt.imshow(img)
    
    # Reshape the image, adding one dimension at the end
    img = img.reshape(1, 28, 28, 1)
    
    # Recast and normalize
    img = img.astype('float32')
    img /= 255
    
    # Get prediction
    prediction = model.predict(img)
    
    # Get highest value index
    prediction = np.argmax(prediction)
    
    print('The prediction for ', image, 'is :', prediction)